# BUILDING CUSTOMER DIMENSION TABLE

# Import libraries 

In [0]:
import pyspark.sql.functions as F
from delta.tables import DeltaTable
from pyspark.sql.functions import trim, col
from pyspark.sql.types import StringType, IntegerType, DateType
from pyspark.sql.window import Window
import logging

# Logging configuration


In [0]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger("silver_crm_cust_info")
logger.setLevel(logging.INFO)

# Define Allowed Prefixes CONFIG

## Later you can change:
## ALLOWED_PREFIXES = ["AW000", "CX000", "BX000"]

In [0]:
ALLOWED_PREFIXES = ["AW000"]
EXPECTED_LENGTH = 10

# Mapping/rules 

In [0]:



# Rename mapping

RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_key",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "create_date",
}

# Transfomations / All columns arrive as string

## Trimming columns

In [0]:
def trim_string_columns(df):
    logger.info("Running trim_string_columns")
    for field in df.schema.fields:
        if isinstance(field.dataType, StringType):
            df = df.withColumn(field.name, trim(col(field.name)))
    return df

## Renaming columns

In [0]:
def rename_columns(df):
    logger.info("Running rename_columns")
    logger.info(f"Columns before rename: {df.columns}")

    for old_name, new_name in RENAME_MAP.items():
        if old_name in df.columns:
            df = df.withColumnRenamed(old_name, new_name)

    logger.info(f"Columns after rename: {df.columns}")
    return df

## Casting columns

In [0]:
def cast_columns(df):
    logger.info("Running cast_columns")

    df = df \
        .withColumn("customer_id", col("customer_id").cast(IntegerType())) \
        .withColumn("create_date", col("create_date").cast(DateType()))

    logger.info("Schema after casting:")
    df.printSchema()

    return df

## Stadardization/Normalization
## for columns: 'marital_status' AND 'gender' 

In [0]:
def standardize_categorical(df):
    logger.info("Running standardize_categorical")

    df = df \
        .withColumn(
            "marital_status",
            F.when(F.upper(F.col("marital_status")).isin("S", "SINGLE"), "Single")
             .when(F.upper(F.col("marital_status")).isin("M", "MARRIED"), "Married")
             .otherwise(None)
        ) \
        .withColumn(
            "gender",
            F.when(F.upper(F.col("gender")).isin("M", "MALE", "1"), "Male")
             .when(F.upper(F.col("gender")).isin("F", "FEMALE", "2"), "Female")
             .otherwise(None)
        )

    return df

# MAIN FLOW

In [0]:
# Main transformation flow

logger.info("Reading Bronze table: workspace.bronze.crm_cust_info")

df = spark.table("workspace.bronze.crm_cust_info")

df = trim_string_columns(df)
df = rename_columns(df)
df = cast_columns(df)
df = standardize_categorical(df)


df = validate_not_null_business_key(
    df,
    key_col="customer_key",
    quarantine_table="workspace.quarantine.crm_cust_info"
)


logger.info("Transformation completed successfully")




Focus on the business key and check for duplicates and nulls, if exist, solve it with counting a row through items and keep those who have most items in a row. If there are dups take the most complite item and latest on date.

#Business Key Enforcement


# Validate null function

What this function does:

Validates a business key column.
Defines invalid rows as: NULL and Empty string ""

Splits the dataframe into
df_invalid → rows that fail validation
df_valid → rows that pass validation

Prints counts for verification.
Returns only valid rows for further processing.

In [0]:
def validate_not_null_business_key(df,key_col,quarantine_table):

    # identify invalid rows
    invalid_condition = (
        F.col(key_col).isNull() |
        (F.col(key_col)=="")
    )

    # separate valid rows
    df_invalid = df.filter(invalid_condition)
    df_valid = df.filter(~invalid_condition)

    invalid_exists = df_invalid.limit(1).count() > 0

    # write invalid to quarantine
    if invalid_exists:

        df_invalid = (
            df_invalid
            .withColumn("rejection_reason", F.lit("null_or_empty_business_key"))
            .withColumn("rejection_stage", F.lit("silver_null_validation"))
            .withColumn("processed_timestamp",F.current_timestamp())

        )
        # We get access to the existing table / run operations like MERGE, UPDATE, or DELETE on it.
        delta_table = DeltaTable.forName(spark, quarantine_table)
        
        # idempotency
        
        #Compare the existing quarantine table (target) with the new invalid rows (source), and treat them as the same record if:
        #They have the same business key OR both business keys are NULL And they have the same rejection_stage
        
        (
        delta_table.alias("target")
        .merge(
            df_invalid.alias("source"),
            f"""
            (
            target.{key_col} = source.{key_col}
            OR (target.{key_col} IS NULL AND source.{key_col} IS NULL)
            )
            AND target.rejection_stage = source.rejection_stage
            """
    )
    .whenNotMatchedInsertAll()
    .execute()
)
        

    # return clean rows
    return df_valid

   




In [0]:
df_clean = validate_not_null_business_key(
    df_test,
    "customer_key",
    "workspace.quarantine.crm_cust_info"
)

In [0]:
%sql
SELECT * FROM workspace.quarantine.crm_cust_info;

In [0]:
%sql
SELECT customer_key, COUNT(*)
FROM workspace.quarantine.crm_cust_info
GROUP BY customer_key;

## A business key is the value that identifies something uniquely in real life.

## 1 Step: Check NULL business keys


## Format check: validate the customer_key,if it starts with expected letters and length and invalid data send to quaratine


In [0]:
logger.info("Starting business key format validation")

# Condition: valid prefix
prefix_condition = F.lit(False)
for p in ALLOWED_PREFIXES:
    prefix_condition = prefix_condition | F.col("customer_key").startswith(p)

# Condition: correct length
length_condition = F.length(F.col("customer_key")) == EXPECTED_LENGTH

# Condition: numeric suffix (after first 5 chars)
numeric_condition = F.col("customer_key").substr(6, EXPECTED_LENGTH - 5).rlike("^[0-9]+$")

# Combine all format conditions
valid_format_condition = prefix_condition & length_condition & numeric_condition

df_invalid_format = df.filter(~valid_format_condition)
df_valid_format = df.filter(valid_format_condition)

invalid_format_count = df_invalid_format.count()
logger.info(f"Invalid format rows found: {invalid_format_count}")






# Quarantine Invalid Format Rows
if invalid_format_count > 0:
    df_invalid_format_with_reason = (
        df_invalid_format
        .withColumn("rejection_reason", F.lit("Invalid business key format"))
        .withColumn("rejection_stage", F.lit("silver_business_key_format_validation"))
        .withColumn("processed_timestamp", F.current_timestamp())
    )

    (
        df_invalid_format_with_reason
        .write
        .format("delta")
        .mode("append")
        .saveAsTable("workspace.quarantine.crm_cust_info")
    )

    logger.info("Invalid format rows written to quarantine")

    # Continue Only With Valid Rows
    df = df_valid_format

In [0]:
%sql
SELECT * from workspace.quarantine.crm_cust_info;


## Step 2: Detect duplicate business keys


In [0]:

duplicate_keys = (
    df.groupBy("customer_key").count().filter(F.col("count") > 1))


dup_key_count = duplicate_keys.count()
logger.info(f"Duplicate customer_key count: {dup_key_count}")
#print(dup_key_count)

## Step 3: Create completeness score (safer ranking logic)

In [0]:
important_cols = [
    "first_name",
    "last_name",
    "marital_status",
    "gender"
]

df = df.withColumn(
    "completeness_score",
    sum(F.col(c).isNotNull().cast("int")
        for c in important_cols)
)

logger.info("Completeness score column created")

## Step 4: Apply deterministic ranking

In [0]:

w = Window.partitionBy("customer_key").orderBy(
    F.col("completeness_score").desc(),
    F.col("create_date").desc_nulls_last(),
    F.col("customer_id").desc_nulls_last()
)

df_ranked = df.withColumn("rn", F.row_number().over(w))

logger.info("Ranking applied using completeness + create_date + customer_id")

In [0]:
df_winners = df_ranked.filter(F.col("rn") == 1)
df_duplicates = df_ranked.filter(F.col("rn") > 1)

if df_duplicates.limit(1).count() > 0:

    logger.info("Duplicate rows detected. Writing to quarantine.")
#display(df_duplicates)
display(df_winners)

# Duplicate Quarantine Block


In [0]:
if dup_rows_count > 0:
    df_duplicates_with_reason = (
        df_duplicates
        .drop("completeness_score", "rn")
        .withColumn("rejection_reason", F.lit("Duplicate business key"))
        .withColumn("rejection_stage", F.lit("silver_business_key_duplicate_resolution"))
        .withColumn("processed_timestamp", F.current_timestamp())
    )

    (
        df_duplicates_with_reason
        .write
        .format("delta")
        .mode("append")
        .saveAsTable("workspace.quarantine.crm_cust_info")
    )

    logger.info("Duplicate rows written to quarantine")

    display(df_winners)
    

## Step 5: Keep best record per business key


In [0]:
df_silver = df_winners \
    .drop("rn", "completeness_score") \
    .drop("source", "source_file", "ingest_timestamp")

logger.info("Duplicate resolution completed")
display(df_silver)

## Final validation

In [0]:
remaining_dups = (
    df_silver.groupBy("customer_key")
             .count()
             .filter(F.col("count") > 1)
             .count()
)

logger.info(f"Remaining duplicates after resolution:---------> {remaining_dups}")

if remaining_dups > 0:
    logger.error("Duplicate keys still exist after resolution.")
    raise Exception("Silver DQ FAILED: Duplicate keys remain")

logger.info("Business key enforcement completed successfully")

In [0]:
%sql
SELECT * FROM workspace.silver.crm_customers

In [0]:
%sql
select * from workspace.quarantine.crm_cust_info